In [29]:
!pip install -q PyPDF2
!pip install -q Levenshtein

In [12]:
from PyPDF2 import PdfReader

reader = PdfReader("./blikk.pdf")

extracted_text = ""
pages = []

for page in reader.pages:
    extracted_text += page.extract_text() + "\n" 
    pages.append(page.extract_text())


In [13]:
import tiktoken

enc = tiktoken.encoding_for_model("gpt-4")
token_count = len(enc.encode(extracted_text))
print(f'Price per document: {(token_count*10.00/10**6+token_count*30/10**6)*380} in HUF')

Price per document: 210.0944 in HUF


In [14]:
import openai
from dotenv import load_dotenv
import os
load_dotenv(override=True)
OPENAI_API_KEY = os.getenv('OPENAI_API_KEY')

openai.api_key = OPENAI_API_KEY


In [15]:
from openai import OpenAI
from langchain import chat_models
from langchain.prompts import ChatPromptTemplate

example_text = pages[0]

llm_gpt4 = chat_models.ChatOpenAI(model_name="gpt-4-turbo", temperature=0.0)
prompt = '''Your task will be to format this text with the following rules in mind.
It is extracted from a pdf with multiple articles on the page. The order of the text might be mixed up.
Fix the order, group texts by article. Group articles in the following json format: 
[{title:"Some title", content:"The content of the article word by word."\},...] for each article!
Make sure to remove "-" and errors that might be present but pay attention to preserve the original text word for word. 
Based on my calculations there are over {article_count} articles you need to extract
DO ALL THE ARTICLES\n\n
'''

template = ChatPromptTemplate.from_template(
        '''{prompt}{article}'''
    )
chain = template | llm_gpt4
response = chain.invoke({"article": example_text, "prompt":prompt, "article_count": 7}).content


<>:8: SyntaxWarning: invalid escape sequence '\}'
<>:8: SyntaxWarning: invalid escape sequence '\}'
C:\Users\takat\AppData\Local\Temp\ipykernel_26076\2417759395.py:8: SyntaxWarning: invalid escape sequence '\}'
  prompt = '''Your task will be to format this text with the following rules in mind.


In [21]:
print(len(example_text))
print(example_text)

17735
„Nyugodj békében2024. április 10., szerda  www.blikk.hu  4 AKTUÁLIS
HOLNAP
NYERJEN10
EZRET!BUDAPEST /emdash.cap Életének ta-
lán legnagyobb álmát si-
került valóra váltania Csi-re Zsuzsinak, és talán sen-
ki nem sejtette, hogy nem 
sokkal azután, hogy szin-te bakancslistája végé-re ért, bekövetkezik a tra-gédia. A Blikk az elsők kö-
zött írta meg Zsuzsi törté-
netét: a lány két szívátülte-
tésen is átesett, ám betegsé-
ge miatt szervezete a har-
madik szívet sem fogadta be. Több transzplantáció-
ra már nem volt lehetőség, 
az orvosok pedig közölték 
vele, hogy nem tudni, hogy 
hetek vagy hónapok van-nak még hátra az életéből, de készüljön fel a megvál-
toztathatatlanra.
Zsuzsi nem adta fel és ba-
kancslistát készített: sorba 
szedte vágyait, álmait, ami-
ket még szeretett volna va-lóra váltani. Nem kell nagy 
dolgokra gondolni: az Ope-
raház és a Nemzeti Színház 
meglátogatása, találkozni a Halott Pénzzel, eljutni az állatkertbe, megsimogat-
ni egy lovat, repülőn utaz-Teljesítet

In [17]:
response

'```json\n[\n    {\n        "title": "Teljesítette álmait a bakancslistás Zsuzsi, majd elhunyt",\n        "content": "Életének talán legnagyobb álmát sikerült valóra váltania Csire Zsuzsinak, és talán senki nem sejtette, hogy nem sokkal azután, hogy szinte bakancslistája végére ért, bekövetkezik a tragédia. A Blikk az elsők között írta meg Zsuzsi történetét: a lány két szívátültetésen is átesett, ám betegsége miatt szervezete a harmadik szívet sem fogadta be. Több transzplantációra már nem volt lehetőség, az orvosok pedig közölték vele, hogy nem tudni, hogy hetek vagy hónapok vannak még hátra az életéből, de készüljön fel a megváltoztathatatlanra. Zsuzsi nem adta fel és bakancslistát készített: sorba szedte vágyait, álmait, amiket még szeretett volna valóra váltani. Nem kell nagy dolgokra gondolni: az Operaház és a Nemzeti Színház meglátogatása, találkozni a Halott Pénzzel, eljutni az állatkertbe, megsimogatni egy lovat, repülőn utazni, és talán a legnagyobb: úszni egyet a tengerben. A

In [18]:
formatted = response.replace('```json\n', '').replace('```', '')

In [19]:
import json
parsed_response = json.loads(formatted)
print(parsed_response)

# Now, let's save this dictionary to a file
file_path = "./parsed_response.json"
with open(file_path, "w") as file:
    json.dump(parsed_response, file)

file_path

[{'title': 'Teljesítette álmait a bakancslistás Zsuzsi, majd elhunyt', 'content': 'Életének talán legnagyobb álmát sikerült valóra váltania Csire Zsuzsinak, és talán senki nem sejtette, hogy nem sokkal azután, hogy szinte bakancslistája végére ért, bekövetkezik a tragédia. A Blikk az elsők között írta meg Zsuzsi történetét: a lány két szívátültetésen is átesett, ám betegsége miatt szervezete a harmadik szívet sem fogadta be. Több transzplantációra már nem volt lehetőség, az orvosok pedig közölték vele, hogy nem tudni, hogy hetek vagy hónapok vannak még hátra az életéből, de készüljön fel a megváltoztathatatlanra. Zsuzsi nem adta fel és bakancslistát készített: sorba szedte vágyait, álmait, amiket még szeretett volna valóra váltani. Nem kell nagy dolgokra gondolni: az Operaház és a Nemzeti Színház meglátogatása, találkozni a Halott Pénzzel, eljutni az állatkertbe, megsimogatni egy lovat, repülőn utazni, és talán a legnagyobb: úszni egyet a tengerben. Az elmúlt fél évben az egész ország 

'./parsed_response.json'

In [25]:
text_count = 0
for item in parsed_response:
    print(f"Title:  {item['title']}")
    print(f"Content: {item['content']}")
    text_count += len(item['content'])
print(f"Total text count: {text_count}")

Title:  Teljesítette álmait a bakancslistás Zsuzsi, majd elhunyt
Content: Életének talán legnagyobb álmát sikerült valóra váltania Csire Zsuzsinak, és talán senki nem sejtette, hogy nem sokkal azután, hogy szinte bakancslistája végére ért, bekövetkezik a tragédia. A Blikk az elsők között írta meg Zsuzsi történetét: a lány két szívátültetésen is átesett, ám betegsége miatt szervezete a harmadik szívet sem fogadta be. Több transzplantációra már nem volt lehetőség, az orvosok pedig közölték vele, hogy nem tudni, hogy hetek vagy hónapok vannak még hátra az életéből, de készüljön fel a megváltoztathatatlanra. Zsuzsi nem adta fel és bakancslistát készített: sorba szedte vágyait, álmait, amiket még szeretett volna valóra váltani. Nem kell nagy dolgokra gondolni: az Operaház és a Nemzeti Színház meglátogatása, találkozni a Halott Pénzzel, eljutni az állatkertbe, megsimogatni egy lovat, repülőn utazni, és talán a legnagyobb: úszni egyet a tengerben. Az elmúlt fél évben az egész ország összefogo

In [31]:
sample = '''Életének talán legnagyobb álmát sikerült valóra váltania Csire Zsuzsinak, és talán senki nem sejtette, hogy nem sokkal azután, hogy szinte bakancslistája végére ért, bekövetkezik a tragédia. A Blikk az elsők között írta meg Zsuzsi történetét: a lány két szívátültetésen is átesett, ám beteg'''
generated = parsed_response[0]['content']
truncated_generated = generated[:len(sample)]

from Levenshtein import distance

string_distance = distance(sample, truncated_generated)
print(f"Error in sample compared to generation: {string_distance} (lower is better)")

Error in sample compared to generation: 0 (lower is better)
